<a href="https://colab.research.google.com/github/ramgopalvarmaar/google-colab-moviepy/blob/main/google-colab-moviepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.kill(os.getpid(), 9)

In [18]:
import requests
from webstories import Story
from bs4 import BeautifulSoup
from moviepy.editor import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import concurrent.futures
import re
from youtube_uploader_selenium import YouTubeUploader
import json
import os
import random
from PIL import Image, ImageOps
from flask_cors import CORS, cross_origin
from flask import Flask, jsonify, request, Response, render_template
from json_database import JsonDatabase
from datetime import datetime
import numpy as np
from io import BytesIO
from pathlib import Path
import shutil

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

db_path = "/Users/rrajen5/Downloads/webstory/assets/videos.db"

formatter = {"PNG": "RGBA", "JPEG": "RGB"}

audio_files = ['audio0.mp3', 'audio1.mp3', 'audio2.mp3', 'audio3.mp3', 'audio4.mp3', 
	'audio5.mp3', 'audio6.mp3', 'audio7.mp3', 'audio8.mp3', 'audio9.mp3', 'audio10.mp3',
	 'audio11.mp3', 'audio12.mp3', 'audio13.mp3','audio14.mp3', 'audio15.mp3', 'audio16.mp3', 'audio17.mp3','audio18.mp3',
'audio19.mp3', 'audio20.mp3', 'audio21.mp3', 'audio22.mp3', 'audio23.mp3', 'audio24.mp3', 'audio25.mp3', 'audio26.mp3',
'audio27.mp3']

tech_hosts = ['www.gadgetsnow.com', 'tech.hindustantimes.com']

supportedhosts = [
'economictimes.indiatimes.com',
'timesofindia.indiatimes.com',
'www.indiatimes.com', 
'www.gadgetsnow.com',
'recipes.timesofindia.com',
'www.pinkvilla.com',
'photogallery.indiatimes.com',
'www.hindustantimes.com',
'tech.hindustantimes.com',
'auto.hindustantimes.com'
]

newsize = (1080,1920)

def resize_create_image_clip(url):
	response_content = requests.get(url, stream=True, headers=headers).content
	img_bytes = BytesIO(response_content)
	img = Image.open(img_bytes)

	#img = Image.open(requests.get(url, stream=True, headers=headers).raw)
	im = ImageOps.pad(img, newsize, color='black')
	np_array = np.array(im)
	return ImageClip(np_array)

def is_grey_scale(url):
    img = Image.open(requests.get(url, stream=True, headers=headers).raw).convert('RGB')
    w, h = img.size
    for i in range(w):
        for j in range(h):
            r, g, b = img.getpixel((i,j))
            if r != g != b: 
                return False
    return True


def check_year_in_string(string):
    pattern = r'\b\d{4}\b'
    match = re.search(pattern, string)
    if match:
        return True
    else:
        return False

def overlay_text_on_clip(clip, finaltext):
	clip_width, clip_height = clip.size
	screensize = (clip_width, clip_height/4.5)
	text_clip = TextClip(txt=finaltext.upper(),
                     kerning=-2, 
                     interline=-1, 
                     size = screensize,
                     method='caption',
                     font="Arial-Bold",
                     fontsize = 32,
                     color="white").set_start(0.30)
	text_clip = text_clip.set_position('center')

	text_clip_width, text_clip_height = text_clip.size
	color_clip = ColorClip(size=(int(text_clip_width*1.1), int(text_clip_height*1.2)), color=(0, 0, 0))
	color_clip = color_clip.set_opacity(.6)

	clip_to_overlay = CompositeVideoClip([color_clip, text_clip])

	width, height = clip_to_overlay.size
	clip_to_overlay = clip_to_overlay.set_pos(('center', 1200))
	clip_to_overlay = clip_to_overlay.set_start(0.20)

	return CompositeVideoClip([clip, clip_to_overlay])


url = "https://photogallery.indiatimes.com/web-stories/ester-expsito-sweeps-the-internet-in-style/photostory/98701497.cms"
contains_grey_scale_image = False
print("scraping")

print(url)
print("contains_grey_scale_image - ",contains_grey_scale_image)

html=requests.get(url, headers=headers).text
story = Story(html)
storyDataDict = { "data":[]}

storyDataDict["title"] = story.title

story_title = story.title
counter = 0
storyData = {}
clips = []
finaltext = story.title
storyDescription = []
clip_counter = 6

db = JsonDatabase("videos", db_path)
videos_found = db.search_by_value("title", story_title.strip()[:88])
print("Already uploaded - ", len(videos_found) > 0)
if len(videos_found) > 0:
  print("Already uploaded")

if len(videos_found) == 0:
  for page in story.pages:
    soup = BeautifulSoup(page.get_clean_html(), "html.parser")
    
    story_video = soup.find("amp-video")
    
    image = soup.find("amp-img")
    if image == None:
      continue

    if image['src'].endswith(".webp"):
      continue

    if contains_grey_scale_image:
      if is_grey_scale(image['src']):
        continue

    storyData["image"+str(counter)] = image['src']

    textOverlay = soup.find("div", class_="description")
    if textOverlay != None:
      storyData["text"+str(counter)] = textOverlay.text.strip()
      finaltext = textOverlay.text.strip()

    textOverlay = soup.find("h1")
    if textOverlay != None:
      storyData["text"+str(counter)] = textOverlay.text.strip()
      finaltext = textOverlay.text.strip()

    if any(host in url for host in ['economictimes.indiatimes.com','timesofindia.indiatimes.com', 'www.gadgetsnow.com', 'recipes.timesofindia.com', 'photogallery.indiatimes.com', 'auto.hindustantimes.com']):
      textOverlay1 = soup.find("h2")
      textOverlay = soup.find("p")
      if textOverlay != None:
        finaltext = textOverlay.text.strip()
        if textOverlay1 != None:
          finaltext = textOverlay1.text.strip() + ": " + finaltext

    if any(host in url for host in ['www.hindustantimes.com', 'tech.hindustantimes.com']):
      textOverlay1 = soup.find("h2")
      textOverlay = soup.find("p")
      if textOverlay1 != None:
        finaltext = textOverlay1.text.strip()
        if textOverlay != None:
          finaltext = textOverlay.text.strip() + " : " + finaltext

    if finaltext.strip() == "":
      finaltext = story.title

    if any(text in finaltext.strip() for text in ['Prachi Malhotra']):
      finaltext = story.title

    if len(finaltext.strip()) < 20 and check_year_in_string(finaltext.strip()):
      finaltext = story.title
    
    print(image['src'])
    print(finaltext)

    try:
      if image['src'].startswith("https"):
        image_clip = resize_create_image_clip(image['src'])
      elif any(host in url for host in ['www.hindustantimes.com']):
        image_clip = resize_create_image_clip("https://www.hindustantimes.com/web-stories/"+image['src'])
      else:
        image_clip = resize_create_image_clip(url+image['src'])
    except Exception as exc:
      error_msg = str(exc)
      print(error_msg)

    proceedToGenerate = False

    if image_clip != None:
      proceedToGenerate = True

    #breakpoint()
    if story_video != None:
      print(story_video)
      proceedToGenerate = True

    if clip_counter > 57:
      proceedToGenerate = False

    if 'pinkvilla' in finaltext.lower():
      proceedToGenerate = False

    if proceedToGenerate:
      if image != None:
        print("Adding image to clip - Total seconds :" + str(clip_counter))
        final_clip = overlay_text_on_clip(image_clip, finaltext)

      if story_video != None:
        print("Adding video to clip - Total seconds :" + str(clip_counter))
        print(story_video.find("source")["src"])

        if any(host in url for host in ['www.hindustantimes.com']):
          if (story_video.find("source")["src"]).startswith("https"):
            video_clip = VideoFileClip(story_video.find("source")["src"])
            video_clip = video_clip.resize(newsize)
          else:
            video_url = "https://www.hindustantimes.com/web-stories/"+story_video.find("source")["src"]
            video_clip = VideoFileClip(video_url)
            video_clip = video_clip.resize(newsize)
        elif any(host in url for host in ['timesofindia.indiatimes.com']):
          if (story_video.find("source")["src"]).startswith("https"):
            video_clip = VideoFileClip(story_video.find("source")["src"])
            video_clip = video_clip.resize(newsize)
          else:
            video_clip = VideoFileClip("https:"+story_video.find("source")["src"])
            video_clip = video_clip.resize(newsize)
        else:
          video_clip = VideoFileClip(story_video.find("source")["src"])
          video_clip = video_clip.resize(newsize)

        final_clip = overlay_text_on_clip(video_clip, finaltext)
        
      if len(finaltext) > 250:
        clip = final_clip.set_duration(5.8)
        clip_counter = clip_counter + 5.8
      elif len(finaltext) > 150:
        clip = final_clip.set_duration(5)
        clip_counter = clip_counter + 5
      elif len(finaltext) > 100:
        clip = final_clip.set_duration(4)
        clip_counter = clip_counter + 4
      elif len(finaltext) > 30:
        clip = final_clip.set_duration(3)
        clip_counter = clip_counter + 3
      else:
        clip = final_clip.set_duration(2)
        clip_counter = clip_counter + 2

      clips.append(clip)
      counter = counter + 1

  storyDataDict["data"].append(storyData)

  if any(host in url for host in tech_hosts): 
    final_subs_clip = ImageClip("/content/assets/gadgetz_subscribe.jpg").set_duration(3)
  else:
    final_subs_clip = ImageClip("/content/assets/subscribe.jpg").set_duration(3)

  final_subs_clip = final_subs_clip.resize(newsize)
  clips.append(final_subs_clip)

  final_clips = [CompositeVideoClip([clip.fx(transfx.slide_out, duration=0.4, side="left")]) for clip in clips]

  concat_clip = concatenate_videoclips(final_clips, method="compose")
  duration = concat_clip.duration

  audio_file = random.choice(audio_files)
  print(audio_file)
  audio_file_clip = AudioFileClip("/content/assets/"+audio_file)
  audio_clip = audio_file_clip.set_duration(duration)
  concat_clip = concat_clip.set_audio(audio_clip)

  final_video_name = re.sub('[^a-zA-Z0-9 \n\.]', '', story_title.strip())
  final_video_name = final_video_name + ".mp4"
  isVideoGenerated = True
  try:
    concat_clip.write_videofile(final_video_name, fps=12)
  except Exception as exc:
    error_msg = str(exc)
    if "could not broadcast input array from shape" in error_msg:
      print("Error due to greyscale image hence retrying ...")
      isVideoGenerated = False
      print("Failed due to Greyscale")
  else:
      isVideoGenerated = True

  if isVideoGenerated:
    metadata = {}
    metadata["title"] = story_title.strip()[:88]

    now = datetime.now()
    metadata["uploaded_date"] = now.strftime("%d/%m/%Y %H:%M:%S")
    print("metadata")
    print(metadata)

    if any(host in url for host in tech_hosts):
      video_output_folder = "video_output/" + now.strftime('%Y-%m-%d')
      Path(video_output_folder).mkdir(parents=True, exist_ok=True)
      shutil.move(final_video_name, video_output_folder+"/"+final_video_name)

    else:	
      try:
        uploader = YouTubeUploader(final_video_name)
        was_video_uploaded, video_id = uploader.upload()
        print(video_id)
      except Exception as exc:
        print("Couldnot complete uploading", exc)

      os.remove(final_video_name)
      
    print("Upload complete")
    with JsonDatabase("videos", db_path) as db:
      db.add_item(metadata)

scraping
https://photogallery.indiatimes.com/web-stories/ester-expsito-sweeps-the-internet-in-style/photostory/98701497.cms
contains_grey_scale_image -  False
Already uploaded -  False
https://static.langimg.com/photo/98701546/98701546.jpg?imgsize=67646
​Ester Expósito sweeps the internet in style ​
Adding image to clip - Total seconds :6
https://static.langimg.com/photo/98701544/98701544.jpg?imgsize=54692
Knows how to win hearts: Ester Exposito has a great passion for fashion. She is also an ambassador for Ralph Lauren.
Adding image to clip - Total seconds :9
https://static.langimg.com/photo/98701526/98701526.jpg?imgsize=130882
Ups the hotness quotient: She was in a relationship with Álvaro Rico, who plays Polo in the television series 'Élite'. The two seemed to be together from October 2018 until July 2019 and then went their separate ways.
Adding image to clip - Total seconds :13
https://static.langimg.com/photo/98701520/98701520.jpg?imgsize=94120
Nailed the look with her flawless p

  warnings.warn(str(msg))



audio20.mp3
[MoviePy] >>>> Building video Ester Expsito sweeps the internet in style .mp4
[MoviePy] Writing audio in Ester Expsito sweeps the internet in style TEMP_MPY_wvf_snd.mp3


100%|██████████| 1015/1015 [00:01<00:00, 559.06it/s]

[MoviePy] Done.
[MoviePy] Writing video Ester Expsito sweeps the internet in style .mp4



100%|█████████▉| 552/553 [06:02<00:00,  1.52it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Ester Expsito sweeps the internet in style .mp4 

metadata
{'title': '\u200bEster Expósito sweeps the internet in style \u200b', 'uploaded_date': '18/03/2023 08:14:17'}
Couldnot complete uploading Message: 'geckodriver' executable needs to be in PATH. 

Upload complete


In [13]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
!pip3 install -r requirements.txt